# DATA 512 - Human-Centered Data Science
## Assignment 2
Will Wright 
Q4 2019

## Purpose and Methodology
In this notebook, the concept of bias is explored through data on Wikipedia articles about politicians and data regarding country population. The Wikipedia data is simply a set of IDs for Wikipedia articles related to politicians and their relevant country. After cleaning the data, it is sent over to an R script, which uses the API to a machine learning system called ORES (Objective Revision Evaluation Service) to predict the quality of the articles. All this data is then put together into 6 tables, as required by the assignment.
***

## Load Modules and Set Parameters

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_rows = 500

## Get Data

In [3]:
page_data = pd.read_csv("../data_raw/page_data.csv")
pop_data = pd.read_csv("../data_raw/WPDS_2018_data.csv")

In [4]:
page_data.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [5]:
pop_data.head()

,Geography,Population mid-2018 (millions)
0,AFRICA,"1,284"
1,Algeria,42.7
2,Egypt,97
3,Libya,6.5
4,Morocco,35.2


## Clean Data
As can be seen above, the page_data table has rows within the 'page' column that start with "Template." These are not wikipedia articles and will need to be removed.

In [6]:
# Use regex to remove rows that start with "Template"
page_data = page_data[~page_data.page.str.contains("^Template:.*")]
page_data.head()

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568


Additionally, the pop_data table has some rows for 'Geography' in all-caps to indicate that they are the population for the entire region.  We'll want to retain these values, but put them in a different table.  After looking through the data, I verified that only the region-level rows were in all caps (no cases of 'USA' or similar) and it's safe to use all-caps as a signal to split the table.

In [7]:
pop_region_data = pop_data[pop_data.Geography.str.contains("^[^a-z]*$")] # retain only uppercase strings
pop_country_data = pop_data[~pop_data.Geography.str.contains("^[^a-z]*$")] # retain only lowercase strings

In [8]:
pop_region_data

,Geography,Population mid-2018 (millions)
0,AFRICA,"1,284"
56,NORTHERN AMERICA,365
59,LATIN AMERICA AND THE CARIBBEAN,649
95,ASIA,"4,536"
144,EUROPE,746
189,OCEANIA,41


In [9]:
pop_country_data.head()

,Geography,Population mid-2018 (millions)
1,Algeria,42.7
2,Egypt,97
3,Libya,6.5
4,Morocco,35.2
5,Sudan,41.7


With this data cleaned, we'll then write page_data to a .csv to be read into the 'hcds-a2-bias_ores-rating.R' script.  This is done because neither of the options provided in Python seem to be working (installing ORES or using the API). 

In [10]:
page_data.to_csv("../data_raw/page_data_nonTemplate.csv", index = False)

#### \*\*Generating Article Scores Using ORES Done in R and Written to .csv\*\*
Pick up ORES prediction results and join on the country of the page_data

In [11]:
prediction_data = pd.read_csv("../data_raw/page_data_predictions.csv")

In [12]:
page_predictions = pd.merge(page_data,
                       prediction_data,
                       left_on = "rev_id",
                       right_on = "rev_id",
                       how = "left")

In [13]:
page_predictions.head()

,page,country,rev_id,prediction
0,Bir I of Kanem,Chad,355319463,Stub
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,Stub
2,Yos Por,Cambodia,393822005,Stub
3,Julius Gregr,Czech Republic,395521877,Stub
4,Edvard Gregr,Czech Republic,395526568,Stub


In [14]:
page_predictions.prediction.value_counts()

Stub                                                    24255
Start                                                   14650
C                                                        5856
GA                                                        755
B                                                         755
FA                                                        275
RevisionNotFound                                          146
TextDeleted: Text deleted (datasource.revision.text)        9
Name: prediction, dtype: int64

Looks like there were 146 cases of RevisionNotFound and 9 cases of Text Deleted. These results will be separated into a log of unused articles.

In [15]:
# set aside RevisionNotFound and TextDeleted predictions
page_predictions_unused = page_predictions[page_predictions.prediction.str.contains('RevisionNotFound') |
                                          page_predictions.prediction.str.contains('TextDeleted')]
page_predictions_unused.to_csv("../data_raw/page_predictions_unused.csv", index = False)

# remove from the predictions dataframe
page_predictions = page_predictions[~(page_predictions.prediction.str.contains('RevisionNotFound') |
                                          page_predictions.prediction.str.contains('TextDeleted'))]

In [16]:
page_predictions.prediction.value_counts()

Stub     24255
Start    14650
C         5856
GA         755
B          755
FA         275
Name: prediction, dtype: int64

Add the country population data from the pop_data dataframe

In [17]:
page_predictions_pop_data = pd.merge(page_predictions,
                       pop_data,
                       left_on = "country",
                       right_on = "Geography",
                       how = "left")

Because not all the countries in the page dataset will necessarily map to the countries in the population dataset, let's investigate which countries aren't matching and by how much.

In [18]:
page_predictions_pop_data.country[page_predictions_pop_data['Population mid-2018 (millions)'].isnull()].value_counts()

Czech Republic                      251
Hondura                             187
Palestinian Territory               179
Congo, Dem. Rep. of                 142
Salvadoran                          116
South Korean                         96
Cape Colony                          81
Samoan                               76
Rhodesian                            75
Faroese                              74
Ivorian                              72
Cook Island                          67
Jersey                               61
Guadeloupe                           49
Saint Lucian                         47
Pitcairn Islands                     43
Chechen                              38
East Timorese                        36
Martinique                           34
Swaziland                            31
Saint Kitts and Nevis                30
French Guiana                        27
Montserratian                        27
Guernsey                             25
Omani                                24


In [19]:
pop_data[pop_data.Geography.str.contains("Czech")]

,Geography,Population mid-2018 (millions)
166,Czechia,10.6


In [20]:
pop_data[pop_data.Geography.str.contains("Hondura")]

,Geography,Population mid-2018 (millions)
64,Honduras,9


Just based on the top two offenders, it looks like we could manually map many (if not all) of the exceptions that were caused by minor differences in spelling or naming format.  In these cases "Czechia" is actually a short-term for "Czech Republic" and "Honduras" is the correct spelling of "Hondura". 

However, given that the instructions grant that we can simply set these exceptions aside and avoid this tedious process, I'm going to opt for that route. If the result of this investigation were to have a real impact, obviously this mapping would make a lot more sense.

In [21]:
# set aside cases where there isn't a direct match
page_predictions_pop_data_unused = page_predictions_pop_data[page_predictions_pop_data['Population mid-2018 (millions)'].isnull()]
page_predictions_pop_data_unused.to_csv("../data_raw/wp_wpds_countries-no_match.csv", index = False)

# remove these exceptions from clean data
page_predictions_pop_data = page_predictions_pop_data[~page_predictions_pop_data['Population mid-2018 (millions)'].isnull()]

Finally, we'll make some changes to the dataframe to match the expected schema and save to the data_clean folder

In [22]:
# drop geography and reorder
page_predictions_pop_data = page_predictions_pop_data[['country','page','rev_id','prediction','Population mid-2018 (millions)']]

In [23]:
# rename
page_predictions_pop_data.columns = ['country','article_name','revision_id','article_quality','population_millions']

In [24]:
page_predictions_pop_data.head()

,country,article_name,revision_id,article_quality,population_millions
0,Chad,Bir I of Kanem,355319463,Stub,15.4
2,Cambodia,Yos Por,393822005,Stub,16
5,Canada,Robert Douglas Cook,401577829,Stub,37.2
6,Egypt,List of Grand Viziers of Egypt,442937236,Stub,97
7,Pakistan,Sehba Musharraf,448555418,Stub,200.6


In [25]:
# write to csv
page_predictions_pop_data.to_csv("../data_clean/wp_wpds_politicians_by_country.csv", index = False)

***
## Analysis
In this section, we explore the cleaned and joined data with a focus on the source of possible biases.  In particular, we're interested in uncovering the articles-per-population and the percent of articles within a country that are high quality ('FA' or 'GA' for 'Featured Article' and 'Good Article').  This will be summarized into 6 tables: 
1. **Top 10 countries by coverage**: 10 highest-ranked countries in terms of number of politician articles as a proportion of country population
2. **Bottom 10 countries by coverage**: 10 lowest-ranked countries in terms of number of politician articles as a proportion of country population
3. **Top 10 countries by relative quality**: 10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
4. **Bottom 10 countries by relative quality**: 10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
5. **Geographic regions by coverage**: Ranking of geographic regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population
6. **Geographic regions by relative quality**: Ranking of geographic regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

The general methodology will be to build a single country-centric table with columns for article counts, coverage (in articles per million population), high quality article counts, and percent high quality.  This will then be subset into the first 4 tables and written to .csv.  Next, a geographic region table will be put together similary for the final two tables.

In [26]:
# read data back in (in case starting from here)
page_predictions_pop_data = pd.read_csv("../data_clean/wp_wpds_politicians_by_country.csv")

For tables 1 and 2, we'll need to create a 'coverage' metric. We can calculate this by grouping on 'country', getting a count of articles per country, dividing by the population, and sorting.

In [27]:
# calculate coverage
article_results = page_predictions_pop_data.groupby('country', as_index = False) \
    .agg({'article_name':'count','population_millions':'first'}) \
    .rename(columns={'article_name':'article_count'})

In [28]:
article_results.dtypes

country                object
article_count           int64
population_millions    object
dtype: object

Looks like 'population' is a string that needs to be converted to float to do the calculation

In [29]:
article_results.population_millions[article_results.population_millions.str.contains(",")]

34    1,393.8
69    1,371.3
Name: population_millions, dtype: object

Ah, the presence of a "," in two of the values is the reason it can't be converted so we'll remove the commas, then convert to float and sort.

In [30]:
article_results.population_millions.replace(regex = True, inplace = True, to_replace = r',', value = r'')
article_results['article_coverage'] = article_results['article_count']/article_results['population_millions'].astype(float)
article_results = article_results.sort_values('article_coverage', ascending = False)

In [31]:
article_results.head()

,country,article_count,population_millions,article_coverage
166,Tuvalu,54,0.01,5400.0
115,Nauru,52,0.01,5200.0
135,San Marino,81,0.03,2700.0
108,Monaco,40,0.04,1000.0
93,Liechtenstein,28,0.04,700.0


For the article quality metrics, we'll need to get a count of 'FA' and 'GA' articles per country and then generate the proportion that have either of those ratings.

In [32]:
# calculate quality as 'FA' or 'GA' per country and percent of articles within country that are quality articles
quality_signals = ['FA','GA']

# get count of 'FA' or 'GA' articles per country
article_results['high_quality_count'] = \
    page_predictions_pop_data.groupby('country', as_index = False)['article_quality'] \
    .apply(lambda x: x[x.str.contains('|'.join(quality_signals))].count())

# calculate pct high quality
article_results['high_quality_pct'] = article_results.high_quality_count/article_results.article_count

In [33]:
article_results.sort_values('high_quality_pct', ascending = False).head()

,country,article_count,population_millions,article_coverage,high_quality_count,high_quality_pct
82,"Korea, North",36,25.6,1.406250,7,0.194444
137,Saudi Arabia,118,33.4,3.532934,15,0.127119
104,Mauritania,48,4.5,10.666667,6,0.125000
31,Central African Republic,66,4.7,14.042553,8,0.121212
132,Romania,343,19.5,17.589744,39,0.113703


Now that all the data is in one results table, we can simply divide it into the top/bottom 10 per article coverage and high_quality_pct to generate tables 1-4.

In [34]:
article_results_top10_coverage = article_results.sort_values('article_coverage', ascending = False).head(10)
article_results_bottom10_coverage = article_results.sort_values('article_coverage', ascending = True).head(10)
article_results_top10_quality = article_results.sort_values('high_quality_pct', ascending = False).head(10)
article_results_bottom10_quality = article_results.sort_values('high_quality_pct', ascending = True).head(10)

Results for the first 4 tables:

In [35]:
article_results_top10_coverage

,country,article_count,population_millions,article_coverage,high_quality_count,high_quality_pct
166,Tuvalu,54,0.01,5400.000000,5,0.092593
115,Nauru,52,0.01,5200.000000,0,0.000000
135,San Marino,81,0.03,2700.000000,0,0.000000
108,Monaco,40,0.04,1000.000000,0,0.000000
93,Liechtenstein,28,0.04,700.000000,0,0.000000
161,Tonga,63,0.1,630.000000,0,0.000000
103,Marshall Islands,37,0.06,616.666667,0,0.000000
68,Iceland,201,0.4,502.500000,2,0.009950
3,Andorra,34,0.08,425.000000,0,0.000000
61,Grenada,36,0.1,360.000000,1,0.027778


**Finding:** There seems to be a strong bias where countries with small populations dominate the coverage.  This is likely the result of there always being at least some Wikipedia articles about politicians in every country so the lower the population, the higher the coverage will seem to be.

In [36]:
article_results_bottom10_coverage

,country,article_count,population_millions,article_coverage,high_quality_count,high_quality_pct
69,India,980,1371.3,0.714650,17,0.017347
70,Indonesia,210,265.2,0.791855,10,0.047619
34,China,1130,1393.8,0.810733,41,0.036283
173,Uzbekistan,28,32.9,0.851064,2,0.071429
51,Ethiopia,101,107.5,0.939535,2,0.019802
82,"Korea, North",36,25.6,1.406250,7,0.194444
178,Zambia,25,17.7,1.412429,0,0.000000
159,Thailand,112,66.2,1.691843,3,0.026786
112,Mozambique,58,30.5,1.901639,0,0.000000
13,Bangladesh,319,166.4,1.917067,3,0.009404


**Findings:** The lowest coverage seems to be biased by at least two obvious features: large populations and restrictive authoritarian regimes.  For instance, India and China are in positions 1 and 3 for the lowest coverage have two of the largest populations so to have a proportional set of politician articles per capita, there would need to be many many more articles.  This may be infeasible for a variety of reasons including the effort to get information, having open channels to that information be limited in areas with little infrastructure, or simply having legitimately fewer politicians per capita.  Additionally, we should expect that restrictive authoritarian regimes would contribute to less information and fewer articles in countries such as China and North Korea.

In [37]:
article_results_top10_quality

,country,article_count,population_millions,article_coverage,high_quality_count,high_quality_pct
82,"Korea, North",36,25.6,1.406250,7,0.194444
137,Saudi Arabia,118,33.4,3.532934,15,0.127119
104,Mauritania,48,4.5,10.666667,6,0.125000
31,Central African Republic,66,4.7,14.042553,8,0.121212
132,Romania,343,19.5,17.589744,39,0.113703
166,Tuvalu,54,0.01,5400.000000,5,0.092593
19,Bhutan,33,0.8,41.250000,3,0.090909
44,Dominica,12,0.07,171.428571,1,0.083333
155,Syria,128,18.3,6.994536,10,0.078125
18,Benin,91,11.5,7.913043,7,0.076923


**Findings:** According to these results, North Korea could be reported as having the highest proportion of high-quality articles, but this is likely biased by several uncaptured features.  For instance, we would imagine that a restrictive authoritarian regime would have few articles overall (which is the case for NK) and that countries that genearlly do bad things would be featured more in the news and, hence, their ratio of high quality to total articles would be high.

In [38]:
article_results_bottom10_quality

,country,article_count,population_millions,article_coverage,high_quality_count,high_quality_pct
4,Angola,106,30.4,3.486842,0,0.0
136,Sao Tome and Principe,21,0.2,105.000000,0,0.0
54,Finland,569,5.5,103.454545,0,0.0
178,Zambia,25,17.7,1.412429,0,0.0
90,Lesotho,29,2.3,12.608696,0,0.0
114,Namibia,162,2.5,64.800000,0,0.0
165,Turkmenistan,32,5.9,5.423729,0,0.0
36,Comoros,51,0.8,63.750000,0,0.0
30,Cape Verde,37,0.6,61.666667,0,0.0
11,Bahamas,20,0.4,50.000000,0,0.0


In [39]:
# number of articles with no high quality articles
article_results[article_results['high_quality_pct']==0].shape[0]

38

**Findings:** Here, because 38 of the countries have 0 high quality articles, there isn't really a 'bottom 10' that makes sense unless we pull in another feature to sort on.  Generally speaking, we'd imagine that high quality articles happen more when a country's politicians do more newsworthy things so it might be useful to control for that by pulling in not just the count of articles about politicians, but the overall count of articles in the country or the relative proportion of news articles outside of Wikipedia.  With these sorts of signals, we could say "For countries within the 3rd decile of newsworthiness, here are the top/bottom 10 for high quality percent of articles." 

#### Generating the Country-Region Mapping
The pop_data table is ordered such that all the countries in a region appear just below that all-caps region.  Thus, we'll need to create a mapping from that data, join it on the article_results table, summarize by region, append the pop_region_data, and perform the final calculations and formatting.

After deliberation, I decided to build the mapping manually in Excel. I realize that it's possible to automate by writing a function that would find the matching country in the pop_data table, then simply go up to the nearest uppercase row and that'd reliably provide the region, but that seems overly-tedious for something that can be done in 3m manually. I understand that's not in the spirit of the course, but I'm a man of practicality and context.  If there were stakes outside of personal grades, obviously this would get more attention.

In [40]:
country_region_mapping = pd.read_csv("../data_raw/country_region_mapping.csv")

In [41]:
# simply a table of country and region
country_region_mapping.head()

,country,region
0,Algeria,AFRICA
1,Egypt,AFRICA
2,Libya,AFRICA
3,Morocco,AFRICA
4,Sudan,AFRICA


In [42]:
country_region_mapping.tail()

,country,region
196,Samoa,OCEANIA
197,Solomon Islands,OCEANIA
198,Tonga,OCEANIA
199,Tuvalu,OCEANIA
200,Vanuatu,OCEANIA


In [43]:
# append region to article_results
article_results_regionDetails = pd.merge(article_results,
                                         country_region_mapping,
                                         left_on = "country",
                                         right_on = "country",
                                         how = "left")

In [44]:
# summarize article count and high_quality_count by region
article_results_regionSummary = article_results_regionDetails.groupby('region', as_index = False) \
    .agg({'article_count':'sum','high_quality_count':'sum'})

# join in population data
article_results_regionSummary = pd.merge(article_results_regionSummary,
                                        pop_region_data,
                                        left_on = "region",
                                        right_on = "Geography",
                                        how = "left")

# drop 'Geography' and rename population
article_results_regionSummary = article_results_regionSummary[['region','article_count','Population mid-2018 (millions)', 'high_quality_count']]
article_results_regionSummary.columns = ['region','article_count','population_millions','high_quality_count']

# like with the country-centric data, remove ","s in the population data and convert to float
article_results_regionSummary.population_millions.replace(regex = True, inplace = True, to_replace = r',', value = r'')

# calculate article_coverage and high_quality_pct
article_results_regionSummary['article_coverage'] = article_results_regionSummary.article_count/article_results_regionSummary.population_millions.astype(float)
article_results_regionSummary['high_quality_pct'] = article_results_regionSummary.high_quality_count/article_results_regionSummary.article_count

In [45]:
article_results_regionSummary

,region,article_count,population_millions,high_quality_count,article_coverage,high_quality_pct
0,AFRICA,6851,1284,125,5.335670,0.018246
1,ASIA,11531,4536,310,2.542108,0.026884
2,EUROPE,15864,746,322,21.265416,0.020298
3,LATIN AMERICA AND THE CARIBBEAN,5169,649,69,7.964561,0.013349
4,NORTHERN AMERICA,1921,365,99,5.263014,0.051536
5,OCEANIA,3128,41,66,76.292683,0.021100


To finish up, we'll just need to create two tables descending on coverage and high_quality_pct (like we did with countries)

In [46]:
article_results_regionSummary_sortDesc_coverage = article_results_regionSummary.sort_values('article_coverage', ascending = False)
article_results_regionSummary_sortDesc_quality = article_results_regionSummary.sort_values('high_quality_pct', ascending = False)

In [47]:
article_results_regionSummary_sortDesc_coverage

,region,article_count,population_millions,high_quality_count,article_coverage,high_quality_pct
5,OCEANIA,3128,41,66,76.292683,0.021100
2,EUROPE,15864,746,322,21.265416,0.020298
3,LATIN AMERICA AND THE CARIBBEAN,5169,649,69,7.964561,0.013349
0,AFRICA,6851,1284,125,5.335670,0.018246
4,NORTHERN AMERICA,1921,365,99,5.263014,0.051536
1,ASIA,11531,4536,310,2.542108,0.026884


In [48]:
article_results_regionSummary_sortDesc_quality # same table, but just sorted on quality

,region,article_count,population_millions,high_quality_count,article_coverage,high_quality_pct
4,NORTHERN AMERICA,1921,365,99,5.263014,0.051536
1,ASIA,11531,4536,310,2.542108,0.026884
5,OCEANIA,3128,41,66,76.292683,0.021100
2,EUROPE,15864,746,322,21.265416,0.020298
0,AFRICA,6851,1284,125,5.335670,0.018246
3,LATIN AMERICA AND THE CARIBBEAN,5169,649,69,7.964561,0.013349


Now, all that's left is to write the 6 tables we've generated to .csv

In [49]:
article_results_top10_coverage.to_csv("../results/article_results_top10_coverage.csv", index = False)
article_results_bottom10_coverage.to_csv("../results/article_results_bottom10_coverage.csv", index = False)
article_results_top10_quality.to_csv("../results/article_results_top10_quality.csv", index = False)
article_results_bottom10_quality.to_csv("../results/article_results_bottom10_quality.csv", index = False)
article_results_regionSummary_sortDesc_coverage.to_csv("../results/article_results_regionSummary_sortDesc_coverage.csv", index = False)
article_results_regionSummary_sortDesc_quality.to_csv("../results/article_results_regionSummary_sortDesc_quality.csv", index = False)

***
## Reflection
Both the sources of data and the steps taken to clean opened the results of this analysis to potential bias. As it stands, if I were a manager getting the tables of this analysis to use for decision making, I'd have a tough time taking it seriously and would need my team to try and capture the sources of bias or find strategies to mitigate the impact of the bias.  

Regarding the data sources, while I mostly trust the population data (having come from the Population Reference Bureau), those figures are estimates based on models where we don't really have any insight into the true quality and it's possible that the population data is biased by some availability-of-information differences in countries. The more-concerning data source is the politician-articles-by-country data since it's only English language (open to the bias of being English-speaking-country-centric) and we don't have insight into how reliable the methodology used was in getting a comprehensive list of articles.

Regarding the bias from the data processing, many opportunities for bias to play a role were introduced. For instance, this analysis required that we only look at the sum of 'FA'- and 'GA'-quality articles to determine 'high quality', but if the goal of the investigation was to truly assess the quality of Wikipedia articles for each country's politicians, then we'd probably want to create some sort of a weighted average across all the quality levels.  Additionally, the ORES API returned some invalid results for the rev_ids sent and these were simply discarded, but it could be the case that those were all high-quality or otherwise biasing.  Futhermore, many countries didn't match between the article data and the population data and these were simply excluded.  These exclusions almost certainly biased the end results.  

